# Satellite Cloud Nowcasting Project: Himawari Imagery Analysis

This notebook demonstrates the entire workflow for the Satellite Cloud Nowcasting project, which uses Himawari-8/9 satellite data for the NTB region.

## Project Overview
- **Domain**: Meteorological Nowcasting
- **Focus**: Cloud Forecasting in Central Indonesia (NTB Region)
- **Satellite**: Himawari-8/9 Band 13 (10.4 µm Infrared)
- **Prediction Horizon**: 0-5 hours

## 1. Setup and Imports

In [ ]:
# Standard library imports
import os
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# Make sure the src module is in the path
sys.path.append(os.getcwd())

# Import project modules
from src.logger import logger
from src.exception import CustomException
from src.components.data_ingestion import DataIngestion
from src.components.data_transformations import DataTransformation
from src.components.model import CloudNowcastingModel
from src.components.data_generator import SatelliteDataGenerator
from src.components.model_evaluation import ModelEvaluator
from src.pipeline.train_pipeline import TrainPipeline
from src.pipeline.predict_pipeline import PredictionPipeline

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')

# Create necessary directories
os.makedirs('artifacts', exist_ok=True)
os.makedirs('data/processed/samples', exist_ok=True)
os.makedirs('logs', exist_ok=True)

## 2. Data Ingestion

In [ ]:
def check_sample_data():
    """
    Check if sample data exists, if not, create synthetic data for testing
    """
    if not os.path.exists("data/train") or not os.path.exists("data/test"):
        logger.info("Sample data not found. Creating synthetic data for demonstration...")

        # Create directories
        os.makedirs("data/train", exist_ok=True)
        os.makedirs("data/test", exist_ok=True)

        # Create synthetic satellite data
        # 576 time steps (equivalent to 4 days at 10-min intervals)
        # 400x400 spatial resolution

        np.random.seed(42)

        # Base pattern (cloud-like structures)
        x = np.linspace(-10, 10, 400)
        y = np.linspace(-10, 10, 400)
        X, Y = np.meshgrid(x, y)
        base = np.exp(-(X**2 + Y**2) / 40)

        # Create time-evolving synthetic data
        train_data = np.zeros((576, 400, 400))
        test_data = np.zeros((144, 400, 400))

        # Generate training data with moving patterns
        for t in range(576):
            # Shift the pattern over time
            shift_x = 5 * np.sin(t/100)
            shift_y = 3 * np.cos(t/120)

            # Create evolving pattern
            pattern = np.roll(np.roll(base, int(shift_x), axis=0), int(shift_y), axis=1)

            # Add some random noise
            noise = 0.05 * np.random.randn(400, 400)

            # Combine pattern and noise
            train_data[t] = pattern + noise

        # Generate test data continuing from training
        for t in range(144):
            # Shift the pattern over time (continuing from training)
            shift_x = 5 * np.sin((t+576)/100)
            shift_y = 3 * np.cos((t+576)/120)

            # Create evolving pattern
            pattern = np.roll(np.roll(base, int(shift_x), axis=0), int(shift_y), axis=1)

            # Add some random noise
            noise = 0.05 * np.random.randn(400, 400)

            # Combine pattern and noise
            test_data[t] = pattern + noise

        # Scale to reasonable brightness temperature range (200-300K)
        train_data = 250 + 50 * train_data
        test_data = 250 + 50 * test_data

        # Save the data
        np.save("data/train/train_data_20250401_0000_to_20250425_0100.npy", train_data)
        np.save("data/test/test_data_20250425_0110_to_20250501_0000.npy", test_data)

        logger.info("Synthetic data created successfully")
        logger.info(f"Training data shape: {train_data.shape}")
        logger.info(f"Test data shape: {test_data.shape}")

        return True
    else:
        # Check if the expected files exist
        train_file = "data/train/train_data_20250401_0000_to_20250425_0100.npy"
        test_file = "data/test/test_data_20250425_0110_to_20250501_0000.npy"

        if not os.path.exists(train_file) or not os.path.exists(test_file):
            logger.info("Expected data files not found. Creating synthetic data...")
            # Remove existing directories and recreate synthetic data
            os.system("rm -rf data/train data/test")
            return check_sample_data()

        logger.info("Sample data found!")
        return True

In [ ]:
# Check for sample data or create it
check_sample_data()

# Load the data
train_data = np.load("data/train/train_data_20250401_0000_to_20250425_0100.npy")
test_data = np.load("data/test/test_data_20250425_0110_to_20250501_0000.npy")

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")

### Visualize Sample Data

In [ ]:
# Visualize a few frames from the training data
plt.figure(figsize=(16, 8))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(train_data[i*10], cmap='jet', vmin=200, vmax=300)
    plt.title(f'Frame {i*10}')
    plt.colorbar(label='Brightness Temperature (K)')
    plt.axis('off')
plt.tight_layout()
plt.suptitle('Sample Frames from Training Data', y=1.05, fontsize=16)
plt.show()

## 3. Data Transformation

In [ ]:
# Initialize data transformation
data_transform = DataTransformation()

# Transform data
logger.info("Starting data transformation...")
start_time = time.time()

(X_train, y_train), (X_test, y_test) = data_transform.initiate_data_transformation(
    train_data, test_data
)

end_time = time.time()
logger.info(f"Data transformation completed in {end_time - start_time:.2f} seconds")

# Print the shapes
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

### Visualize Sample Sequence

In [ ]:
# Visualize a sample input and target sequence
sample_idx = 0

# Save sample sequences for future use
np.save("data/processed/samples/sample_input_sequence.npy", X_train[sample_idx])
np.save("data/processed/samples/sample_target_sequence.npy", y_train[sample_idx])

plt.figure(figsize=(20, 8))

# Plot input sequence
for i in range(X_train.shape[1]):  # For each time step in input sequence
    plt.subplot(2, 6, i+1)
    plt.imshow(X_train[sample_idx, i], cmap='jet')
    plt.title(f'Input t-{X_train.shape[1]-i}')
    plt.colorbar()
    plt.axis('off')

# Plot target sequence
for i in range(y_train.shape[1]):  # For each time step in target sequence
    plt.subplot(2, 6, i+7)
    plt.imshow(y_train[sample_idx, i], cmap='jet')
    plt.title(f'Target t+{i+1}')
    plt.colorbar()
    plt.axis('off')

plt.tight_layout()
plt.suptitle('Sample Input-Target Sequence Pair', y=1.05, fontsize=16)
plt.show()

## 4. Model Building and Training

In [ ]:
# Add channel dimension to input and target sequences if needed
if len(X_train.shape) == 3:
    X_train = np.expand_dims(X_train, axis=-1)
if len(y_train.shape) == 3:
    y_train = np.expand_dims(y_train, axis=-1)
if len(X_test.shape) == 3:
    X_test = np.expand_dims(X_test, axis=-1)
if len(y_test.shape) == 3:
    y_test = np.expand_dims(y_test, axis=-1)

# Check updated shapes
print(f"Updated X_train shape: {X_train.shape}")
print(f"Updated y_train shape: {y_train.shape}")

In [ ]:
# Initialize model
model_builder = CloudNowcastingModel()

# Set input shape based on the actual data dimensions
model_builder.config.input_shape = X_train.shape[1:]
model_builder.config.epochs = 10  # Reduced for demonstration

# Build the model
model = model_builder.build_model()

# Print model summary
model.summary()

In [ ]:
# Training with reduced epochs for demonstration
logger.info("Starting model training...")
start_time = time.time()

# Create a validation split
val_split = 0.1
val_size = int(len(X_train) * val_split)

X_val = X_train[-val_size:]
y_val = y_train[-val_size:]
X_train_final = X_train[:-val_size]
y_train_final = y_train[:-val_size]

# Train the model
history = model_builder.train_model(
    model,
    train_data=(X_train_final, y_train_final),
    valid_data=(X_val, y_val),
    callbacks=[
        # Save best model
        tf.keras.callbacks.ModelCheckpoint(
            'artifacts/best_model.h5',
            save_best_only=True,
            monitor='val_loss'
        ),
        # Early stopping
        tf.keras.callbacks.EarlyStopping(
            patience=5,
            monitor='val_loss',
            restore_best_weights=True
        ),
        # Reduce learning rate
        tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.5,
            patience=2,
            monitor='val_loss',
            min_lr=1e-6
        )
    ]
)

end_time = time.time()
logger.info(f"Model training completed in {end_time - start_time:.2f} seconds")

# Save the final model
model.save('artifacts/final_model.h5')

### Plot Training History

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.savefig('artifacts/training_history.png')
plt.show()

## 5. Model Evaluation

In [ ]:
# Evaluate on test set
logger.info("Evaluating model on test set...")
test_metrics = model.evaluate(X_test, y_test, verbose=1)
metrics = dict(zip(model.metrics_names, test_metrics))

print("\nTest Set Metrics:")
for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

In [ ]:
# Make predictions on a sample from the test set
sample_idx = 5  # Select a random sample from test set
sample_input = X_test[sample_idx:sample_idx+1]  # Add batch dimension

# Make prediction
sample_prediction = model.predict(sample_input)

# Remove batch dimension
sample_input = sample_input[0]
sample_prediction = sample_prediction[0]
sample_ground_truth = y_test[sample_idx]

# Visualize the results
plt.figure(figsize=(18, 12))

# Plot the last frame of input sequence
plt.subplot(3, 6, 1)
plt.imshow(sample_input[-1, :, :, 0], cmap='jet')
plt.title('Last Input Frame (t=0)')
plt.colorbar()
plt.axis('off')

# Plot ground truth and predictions side by side
for i in range(sample_prediction.shape[0]):  # For each prediction time step
    # Ground truth
    plt.subplot(3, 6, i+7)
    plt.imshow(sample_ground_truth[i, :, :, 0], cmap='jet')
    plt.title(f'Ground Truth (t+{i+1})')
    plt.colorbar()
    plt.axis('off')

    # Prediction
    plt.subplot(3, 6, i+13)
    plt.imshow(sample_prediction[i, :, :, 0], cmap='jet')
    plt.title(f'Prediction (t+{i+1})')
    plt.colorbar()
    plt.axis('off')

plt.tight_layout()
plt.savefig('artifacts/prediction_sample.png')
plt.show()

## 6. Prediction Pipeline

In [ ]:
# Save data transformer for prediction pipeline
from src.utils import save_object
save_object('artifacts/data_transformer.pkl', data_transform)

In [ ]:
# Create a prediction pipeline using our latest time steps from test data
try:
    # Get a sample from test data
    sample_input_sequence = X_test[0]  # First sequence in test data

    # Initialize prediction pipeline
    predictor = PredictionPipeline()

    # Make prediction
    logger.info("Making prediction using prediction pipeline...")
    prediction = predictor.predict(sample_input_sequence)

    print(f"Prediction shape: {prediction.shape}")

    # Visualize the prediction
    plt.figure(figsize=(18, 6))

    # Plot the last input frame
    plt.subplot(1, 7, 1)
    plt.imshow(sample_input_sequence[-1, :, :, 0], cmap='jet')
    plt.title('Last Input Frame (t=0)')
    plt.colorbar()
    plt.axis('off')

    # Plot the predicted frames
    for i in range(prediction.shape[0]):
        plt.subplot(1, 7, i+2)
        plt.imshow(prediction[i], cmap='jet')
        plt.title(f'Predicted (t+{i+1})')
        plt.colorbar()
        plt.axis('off')

    plt.tight_layout()
    plt.savefig('artifacts/pipeline_prediction.png')
    plt.show()

except Exception as e:
    print(f"Error in prediction pipeline: {str(e)}")

## 7. Running the Full Training Pipeline (Optional)

This would start the full training pipeline using the `TrainPipeline` class, which includes optimized data generators, mixed precision training, and more advanced features. This might take longer to run, so it's optional.

In [ ]:
# Uncomment to run the full training pipeline
'''
# Initialize the training pipeline
trainer = TrainPipeline(batch_size=4)  # Reduced batch size for memory efficiency

# Start training
logger.info("Starting full training pipeline...")
try:
    metrics, history = trainer.initiate_training()
    print("Training completed successfully!")
    print("\nFinal Metrics:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")
except Exception as e:
    print(f"Error in training pipeline: {str(e)}")
'''

## 8. Conclusion

This notebook has demonstrated the complete workflow for the Cloud Nowcasting project:

1. **Data Ingestion**: Loading and preparing satellite data
2. **Data Transformation**: Preprocessing, normalization, and sequence creation
3. **Model Building**: Creating a ConvLSTM neural network architecture
4. **Model Training**: Training the model with appropriate callbacks
5. **Model Evaluation**: Assessing performance on test data
6. **Prediction Pipeline**: Using the trained model for inference

The trained model can be used for short-term cloud forecasting in the NTB region based on Himawari satellite imagery.